In [1]:
#The goal is to write a script that given a movie name returns similar movies.

#Also, truncate the dataset to top 1000 movies in terms of review count. This should give more robust results.

In [2]:
import numpy as np
import pandas as pd

ratings = pd.read_csv('https://raw.githubusercontent.com/jputrius/ml_intro/refs/heads/main/data/movielens/ratings.csv')
movies = pd.read_csv('http://raw.githubusercontent.com/jputrius/ml_intro/refs/heads/main/data/movielens/movies.csv')

In [47]:
topid = ratings.groupby("movieId").size().sort_values(ascending=False)[0:1000].index
top = movies.set_index("movieId").loc[topid]

Index([   356,    318,    296,    593,   2571,    260,    480,    110,    589,
          527,
       ...
       112183,   5945,  93510,   3556,   3638,  68319,   3869,   5574,   3969,
       106487],
      dtype='int64', name='movieId', length=1000)

In [35]:
ratings = ratings[ratings["movieId"].isin(top.index)]

,userId,movieId,rating,timestamp,rating_count
0,1,1,4.0,964982703,215
1,1,3,4.0,964981247,52
2,1,6,4.0,964982224,102
3,1,47,5.0,964983815,203
4,1,50,5.0,964982931,204
...,...,...,...,...,...
100763,610,134853,3.5,1493845106,43
100780,610,139385,4.5,1493846777,31
100803,610,148626,4.0,1493847175,26
100808,610,152081,4.0,1493846503,32


In [43]:
design = ratings.pivot(index="userId", columns="movieId", values="rating").fillna(2.5)
design_centered = design-design.mean()

In [45]:
X = design_centered.to_numpy()
U,M,V = np.linalg.svd(X, full_matrices=False)
V1 = V[:40].T

In [52]:
def recommendations(movie, data):
    movieid = top[top["title"]==movie].index[0]
    movieidx = np.argwhere(topid==movieid)[0]
    movie_to_compare = data[movieidx]
    
    distance = np.linalg.norm(data - movie_to_compare, axis=1)
    similar_ids = topid[distance.argsort()[:10]]
    similar_movies = top.loc[similar_ids]
    return similar_movies["title"].to_numpy()

In [53]:
print(f"Movies similar to:")
print(f"Toy Story: {recommendations('Toy Story (1995)', V1)}")
print(f"Iron Man: {recommendations('Iron Man (2008)', V1)}")

Movies similar to:
Toy Story: ['Toy Story (1995)' 'Up (2009)' 'GoldenEye (1995)' 'Aliens (1986)'
 'Léon: The Professional (a.k.a. The Professional) (Léon) (1994)'
 'Star Trek II: The Wrath of Khan (1982)' 'Disclosure (1994)'
 'Four Weddings and a Funeral (1994)' 'Incredibles, The (2004)'
 'Fugitive, The (1993)']
Iron Man: ['Iron Man (2008)' 'Breakfast Club, The (1985)' 'Clerks (1994)'
 'Bourne Identity, The (2002)' 'Dumb & Dumber (Dumb and Dumber) (1994)'
 'Star Wars: Episode V - The Empire Strikes Back (1980)'
 'Toy Story 2 (1999)' 'Godfather: Part II, The (1974)' 'Babe (1995)'
 'Four Weddings and a Funeral (1994)']
